In [1]:
import pandas as pd
import numpy as np
from astropy.table import Table
from astropy.io import fits
from astropy.cosmology import FlatLambdaCDM, z_at_value, Planck18
from astropy import units as u
import kcorrect.kcorrect

from vmax_cal import cal_v_vmax_mp
import multiprocessing as mp

$V_{max}$的计算

$$V_{\text{max}}=\int d\Omega f(\theta,\phi)\int_{z_{\text{min}}}^{z_{\text{max}}(\theta,\phi)}dz\frac{dV}{dz}$$

$$z_{\text{max}}(\theta,\phi)=\min\bigl[z_{m,\text{max}}(\theta,\phi),0.05\bigr]$$

$$m_{r,\text{max}}(\theta,\phi)=M_r+\text{DM}\bigl(z_{m,\text{max}}(\theta,\phi)\bigr) +K_r\big(z_{m,\text{max}}(\theta,\phi)\big)$$

In [2]:
sdss_photo_df = pd.read_csv('/Users/abry/Library/CloudStorage/OneDrive-北京大学/Lessons/Lesson_1_2/galaxy/Quenching/mpa_dr4_join_abry.csv')

In [6]:
sdss_mags = sdss_photo_df.loc[0:30, ['dered_u', 'dered_g', 'dered_r', 'dered_i', 'dered_z',]].values
sdss_mag_errs = sdss_photo_df.loc[0:30, ['modelMagErr_u', 'modelMagErr_g', 'modelMagErr_r', 'modelMagErr_i', 'modelMagErr_z',]].values
gal_zs = sdss_photo_df.loc[0:30, 'z'].values

In [7]:
sdss_korr = kcorrect.kcorrect.KcorrectSDSS(abcorrect=True, cosmo=Planck18)
(maggies,ivars) = kcorrect.utils.sdss_asinh_to_maggies(mag=sdss_mags, mag_err=sdss_mag_errs)
coeffs = sdss_korr.fit_coeffs_asinh(redshift = gal_zs, mag=sdss_mags, mag_err=sdss_mag_errs)
ab_mags = sdss_korr.absmag(maggies=maggies, ivar=ivars, redshift=gal_zs, coeffs=coeffs)
ab_mags_r = ab_mags[:,2]

In [27]:
idx = 2
coeff=coeffs[idx]
ab_mag_r=ab_mags_r[idx]
m_max_r=17.77+0.01
def dist_mod(gal_z):
    return 5*np.log10(Planck18.luminosity_distance(gal_z).to(u.pc).value/10)
def minimize_zmax(z):
    return m_max_r - dist_mod(z) - sdss_korr.kcorrect(redshift=z, coeffs=coeff)[2] - ab_mag_r
# minimize_zmax = np.vectorize(minimize_zmax)

In [28]:
minimize_zmax(0.085)

-0.0042095048022758874

In [25]:
from scipy.optimize import fsolve, brentq

In [29]:
brentq(minimize_zmax, 0.02, 0.085)

0.08485173061476103

In [4]:
v_vmax = cal_v_vmax_mp(sdss_mags, sdss_mag_errs, gal_zs)

Calculating V/Vmax...
Calculating V/Vmax...
Calculating V/Vmax...
Calculating V/Vmax...
Calculating V/Vmax...
Calculating V/Vmax...
Calculating V/Vmax...
Calculating V/Vmax...


4it [00:00, 50.17it/s]
4it [00:00, 63.19it/s]
4it [00:00, 77.97it/s]
4it [00:00, 89.44it/s]
4it [00:00, 95.46it/s]
3it [00:00, 119.71it/s]
4it [00:00, 100.40it/s]
4it [00:00, 132.41it/s]


In [4]:
v_vmax_list = []
with mp.Pool(8) as p:
    v_vmax_list .append(p.starmap(cal_v_vmax, 
                                 zip(np.array_split((sdss_mags),8), np.array_split((sdss_mag_errs),8), np.array_split((gal_zs),8))))

Calculating V/Vmax...
Calculating V/Vmax...
Calculating V/Vmax...
Calculating V/Vmax...
Calculating V/Vmax...
Calculating V/Vmax...
Calculating V/Vmax...
Calculating V/Vmax...


In [6]:
v_vmax_list[0]

[array([ 1.        ,  1.        ,  1.00521337,  1.        ]),
 array([ 1.,  1.,  1.,  1.]),
 array([ 1.        ,  1.61146421,  1.        ,  1.        ]),
 array([ 3.89007723,  1.        ,  1.        ,  1.        ]),
 array([ 1.,  1.,  1.,  1.]),
 array([ 1.,  1.,  1.,  1.]),
 array([ 1.,  1.,  1.,  1.]),
 array([ 1.39280265,  1.73478946,  2.36310053])]

In [10]:
np.concatenate((a,b,b), axis=0)

array([1, 2, 3, 4, 5, 4, 5])

In [13]:
list(zip(sdss_mags, sdss_mag_errs, gal_zs))[0]

(array([ 16.91713,  15.3146 ,  14.55245,  14.14041,  13.83297]),
 array([ 0.01122162,  0.00265413,  0.00226598,  0.00226491,  0.00359701]),
 0.02122228)

In [11]:
gal_zs

array([ 0.02122228,  0.06490172,  0.05654794,  0.06215091,  0.06758517,
        0.06727324,  0.03580831,  0.05665967,  0.06251308,  0.03073909])

In [57]:
v_vmax = cal_v_vmax(
    sdss_mags = sdss_mags, sdss_mag_errs = sdss_mag_errs, gal_zs = gal_zs)

Calculating V/Vmax...


In [54]:
v_vmax

array([ 1.        ,  1.        ,  1.00521337,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.61146421,
        1.        ])